<h2><font color="#000000" size=6>Minería de datos</font></h2>
<h1><font color="#000000" size=5>PEC 4 - Random Forest</font></h1>
<br><br>
<div style="text-align: right">
<font color="#000000" size=3>Estudiante: Fernando H. Nasser-Eddine López</font><br>
<font color="#000000" size=3>Máster Universitario en Investigación en Inteligencia Artificial (MUIIA)</font><br>
<font color="#000000" size=3>Mayo 2025</font><br>
</div>

<h2><font color="#000000" size=5>Índice</font></h2><a id="indice"></a>

* [4. Fase de optimización](#section4)
    * [4.1. Implementación de búsqueda exhaustiva de hiperparámetros mediante validación cruzada](#section41)
        * [4.1.1. Optimización del modelo 1 (Random Forrest)](#section411)
        * [4.1.2. Optimización del modelo 2 (Extra Trees)](#section412)
        * [4.1.3. Optimización del modelo 3 (Bagging)](#section413)
    * [4.2. Evaluación de los modelos optimizados](#section42)
* [5. Conclusiones](#section5)

# <font color="#000000"> 4. Fase de optimización</font><a id="section4"></a>

### <font color="#000000">Importación de librerías</font><a id="section11"></a>


En esta sección realizamos la importación de todas las librerías que utilizaremos a lo largo del análisis. Principalmente, usamos pandas para la manipulación de datos, matplotlib y seaborn para visualizaciones, así como scikit-learn para los algoritmos de aprendizaje automático y evaluación de modelos.

In [1]:
# Importación de las librerías necesarias
import time
import warnings
import os
import pickle

import nbimporter
from a_analisis import detect_outliers_comprehensive

import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            confusion_matrix, roc_auc_score, classification_report, 
                            roc_curve, auc)

# Modelos base
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance

# Modelos ensemble
from sklearn.ensemble import (RandomForestClassifier, BaggingClassifier, 
                             ExtraTreesClassifier, AdaBoostClassifier)

# Explicabilidad
import shap

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)

# Evitar warnings innecesarios
warnings.filterwarnings('ignore')

RANDOM_STATE = 42

Para seleccionar los tres mejores modelos ensemble, podemos establecer algunos criterios como:
1. **Rendimiento cuantitativo**: Exactitud, precisión, sensibilidad y F1-Score
2. **Estabilidad**: Diferencia entre rendimiento en entrenamiento y prueba
3. **Interpretabilidad**: Capacidad para proporcionar importancia de características
4. **Versatilidad**: Rendimiento tanto con datos originales como preprocesados

| Modelo | Rendimiento (Datos prep.) | Estabilidad | Interpretabilidad | Versatilidad |
|--------|---------------------------|-------------|-------------------|--------------|
| Random Forest | 100% en todas las métricas | Alta | Alta | Buena (93.56% en original) |
| Bagging-DT | 98.83% exactitud | Media | Media | Buena (94.74% en original) |
| Bagging-LR | 100% en todas las métricas | Alta | Baja | Buena (94.15% en original) |
| Bagging-SVM | 100% en todas las métricas | Alta | Baja | Baja (91.23% en original) |
| Extra Trees | 100% en todas las métricas | Alta | Alta | Muy buena (97.08% en original) |

Basándonos en estos criterios, los tres mejores modelos podrían ser:

1. **Extra Trees**: Rendimiento perfecto con datos preprocesados y el mejor desempeño con datos originales (97.08%), combinado con alta interpretabilidad mediante su ranking de importancia de características. Su mayor capacidad para manejar datos sin preprocesar lo convierte en el modelo más versátil.

2. **Random Forest**: Rendimiento perfecto con datos preprocesados, alta interpretabilidad y buena versatilidad. Proporciona una perspectiva complementaria a Extra Trees en cuanto a importancia de variables.

3. **Bagging con Regresión Logística**: Combina las ventajas del ensemble con un clasificador lineal base, logrando rendimiento perfecto con datos preprocesados y buena versatilidad (94.15% con datos originales). Representa una aproximación distinta a los modelos basados en árboles.

In [2]:
# Cargar modelos para optimización
with open('data/models/models_for_optimization.pkl', 'rb') as f:
    models_data = pickle.load(f)

# Extraer modelos y datos
rf_model = models_data['random_forest']
et_model = models_data['extra_trees']
bagging_lr = models_data['bagging_lr']
X_train_model = models_data['X_train']
X_test_model = models_data['X_test']
y_train_model = models_data['y_train']
y_test_model = models_data['y_test']


## <font color="#000000"> 4.1. Implementación de búsqueda exhaustiva de hiperparámetros mediante validación cruzada</font><a id="section41"></a>

In [3]:
def optimize_model(model_type, param_grid, X_train, y_train, X_test, y_test, random_state=42):
    """
    Optimiza un modelo ensemble buscando la configuración más simple que mantenga 100% de exactitud.
    """
    results = []
    
    # Generamos todas las combinaciones de hiperparámetros
    param_keys = list(param_grid.keys())
    param_values = list(param_grid.values())
    
    # Función para calcular la complejidad del modelo
    def get_complexity(params):
        if model_type == 'RandomForest' or model_type == 'ExtraTrees':
            max_depth = params.get('max_depth')
            max_depth_value = float('inf') if max_depth is None else max_depth
            return params['n_estimators'] * max_depth_value
        elif model_type == 'BaggingLR':
            return params['n_estimators'] * (1 if params.get('max_samples') == 1.0 else params.get('max_samples'))
    
    from itertools import product
    for values in product(*param_values):
        params = dict(zip(param_keys, values))
        
        # Creamos el modelo con los hiperparámetros actuales
        if model_type == 'RandomForest':
            model = RandomForestClassifier(random_state=random_state, **params)
        elif model_type == 'ExtraTrees':
            model = ExtraTreesClassifier(random_state=random_state, **params)
        elif model_type == 'BaggingLR':
            base_estimator = LogisticRegression(max_iter=1000, random_state=random_state)
            model = BaggingClassifier(estimator=base_estimator, random_state=random_state, **params)
        
        # Evaluación con validación cruzada
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
        
        # Evaluación en conjunto de prueba
        model.fit(X_train, y_train)
        test_acc = accuracy_score(y_test, model.predict(X_test))
        
        # Guardamos resultados
        result = params.copy()
        result.update({
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'test_acc': test_acc,
            'complexity': get_complexity(params)
        })
        # Guardamos también los tipos originales de cada parámetro para restaurarlos después
        result['param_types'] = {k: type(v) for k, v in params.items()}
        results.append(result)
    
    # Convertimos a DataFrame
    results_df = pd.DataFrame(results)
    
    # Filtramos modelos que alcanzan 100% de exactitud
    perfect_models = results_df[results_df['test_acc'] == 1.0]
    
    if len(perfect_models) > 0:
        # Ordenamos por complejidad y estabilidad
        perfect_models = perfect_models.sort_values(['complexity', 'cv_std'], ascending=[True, True])
        best_params_row = perfect_models.iloc[0]
        
        # Extraemos y corregimos tipos de los parámetros
        best_params = {}
        param_types = best_params_row['param_types']
        for k in param_keys:
            value = best_params_row[k]
            # Restauramos el tipo original
            if k == 'max_depth' and value is None:
                best_params[k] = None
            elif param_types[k] == int:
                best_params[k] = int(value)
            elif param_types[k] == float:
                best_params[k] = float(value)
            elif param_types[k] == bool:
                best_params[k] = bool(value)
            else:
                best_params[k] = value
        
        # Creamos el mejor modelo con los parámetros corregidos
        if model_type == 'RandomForest':
            best_model = RandomForestClassifier(random_state=random_state, **best_params)
        elif model_type == 'ExtraTrees':
            best_model = ExtraTreesClassifier(random_state=random_state, **best_params)
        elif model_type == 'BaggingLR':
            base_estimator = LogisticRegression(max_iter=1000, random_state=random_state)
            best_model = BaggingClassifier(estimator=base_estimator, random_state=random_state, **best_params)
        
        best_model.fit(X_train, y_train)
    else:
        # Si ningún modelo alcanza 100%, ordenamos por exactitud en prueba
        results_df = results_df.sort_values(['test_acc', 'complexity', 'cv_std'], 
                                          ascending=[False, True, True])
        best_params_row = results_df.iloc[0]
        
        # Extraemos y corregimos tipos de los parámetros
        best_params = {}
        param_types = best_params_row['param_types']
        for k in param_keys:
            value = best_params_row[k]
            # Restauramos el tipo original
            if k == 'max_depth' and value is None:
                best_params[k] = None
            elif param_types[k] == int:
                best_params[k] = int(value)
            elif param_types[k] == float:
                best_params[k] = float(value)
            elif param_types[k] == bool:
                best_params[k] = bool(value)
            else:
                best_params[k] = value
        
        # Creamos el mejor modelo con los parámetros corregidos
        if model_type == 'RandomForest':
            best_model = RandomForestClassifier(random_state=random_state, **best_params)
        elif model_type == 'ExtraTrees':
            best_model = ExtraTreesClassifier(random_state=random_state, **best_params)
        elif model_type == 'BaggingLR':
            base_estimator = LogisticRegression(max_iter=1000, random_state=random_state)
            best_model = BaggingClassifier(estimator=base_estimator, random_state=random_state, **best_params)
        
        best_model.fit(X_train, y_train)
    
    return best_model, best_params, results_df

### <font color="#000000"> 4.1.1. Optimización del modelo 1 (Random Forrest)</font><a id="section421"></a>

In [4]:
# Evaluación de diferentes configuraciones de Random Forest
param_grid_rf = {
    'n_estimators': [10, 25, 50, 100],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Optimización de Random Forest
print("Optimizando Random Forest...")
best_rf, best_rf_params, rf_results = optimize_model(
    'RandomForest', param_grid_rf, X_train_model, y_train_model, X_test_model, y_test_model
)

Optimizando Random Forest...


### <font color="#000000"> 4.1.2. Optimización del modelo 2 (Extra Trees)</font><a id="section422"></a>

In [5]:
# Definimos el grid de hiperparámetros para Extra Trees
param_grid_et = {
    'n_estimators': [10, 25, 50, 100],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Optimización de Extra Trees
print("Optimizando Extra Trees...")
best_et, best_et_params, et_results = optimize_model(
    'ExtraTrees', param_grid_et, X_train_model, y_train_model, X_test_model, y_test_model
)

Optimizando Extra Trees...


### <font color="#000000"> 4.1.3. Optimización del modelo 3 (Bagging)</font><a id="section423"></a>

In [6]:
# Definimos el grid de hiperparámetros para Bagging
param_grid_bagging_lr = {
    'n_estimators': [10, 25, 50, 100],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}

# Optimización de Bagging con Regresión Logística
print("Optimizando Bagging con Regresión Logística...")
best_bagging_lr, best_bagging_lr_params, bagging_lr_results = optimize_model(
    'BaggingLR', param_grid_bagging_lr, X_train_model, y_train_model, X_test_model, y_test_model
)

Optimizando Bagging con Regresión Logística...


In [7]:
# Presentación de resultados
for model_name, params in [("Random Forest", best_rf_params), 
                          ("Extra Trees", best_et_params), 
                          ("Bagging con LR", best_bagging_lr_params)]:
    print(f"\nConfiguración óptima para {model_name}:")
    for param, value in params.items():
        if param in ['cv_mean', 'cv_std', 'test_acc', 'complexity']:
            if param.startswith('cv_') or param == 'test_acc':
                print(f"{param}: {value:.4f}")
            else:
                print(f"{param}: {value:.2f}")
        else:
            print(f"{param}: {value}")


Configuración óptima para Random Forest:
n_estimators: 25
max_depth: 5
min_samples_split: 2
min_samples_leaf: 2

Configuración óptima para Extra Trees:
n_estimators: 25
max_depth: 5
min_samples_split: 10
min_samples_leaf: 1

Configuración óptima para Bagging con LR:
n_estimators: 10
max_samples: 0.5
max_features: 1.0
bootstrap: False
bootstrap_features: False


### <font color="#000000"> 4.1.4. Evaluación de los modelos optimizados</font><a id="section414"></a>

In [8]:
# Evaluación de modelos optimizados
print("# Evaluación de modelos optimizados")

# Función para evaluar y mostrar resultados detallados
def evaluate_optimized_model(model, model_name):
    model.fit(X_train_model, y_train_model)
    
    # Predicciones
    y_pred_train = model.predict(X_train_model)
    y_pred_test = model.predict(X_test_model)
    
    # Métricas en entrenamiento
    acc_train = accuracy_score(y_train_model, y_pred_train)
    
    # Métricas en prueba
    acc_test = accuracy_score(y_test_model, y_pred_test)
    prec = precision_score(y_test_model, y_pred_test)
    rec = recall_score(y_test_model, y_pred_test)
    f1 = f1_score(y_test_model, y_pred_test)
    
    # Crear un DataFrame con los resultados
    results_df = pd.DataFrame({
        'Modelo': [model_name],
        'Accuracy Train': [acc_train],
        'Accuracy Test': [acc_test],
        'Precision': [prec],
        'Recall': [rec],
        'F1-Score': [f1]
    })
    
    # Mostrar el DataFrame
    display(results_df)
    
    # Matriz de confusión
    cm = confusion_matrix(y_test_model, y_pred_test)

    # Crear un DataFrame para la matriz de confusión
    cm_df = pd.DataFrame(cm, 
                        index=['Maligno (0)', 'Benigno (1)'], 
                        columns=['Predicción Maligno (0)', 'Predicción Benigno (1)'])
        
    # Mostrar la matriz de confusión como tabla
    display(cm_df)
    
    return {
        'accuracy_train': acc_train,
        'accuracy_test': acc_test,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'confusion_matrix': cm
    }

# Evaluamos cada modelo optimizado
print("\nRandom Forest optimizado")
print(f"Configuración: n_estimators={best_rf_params['n_estimators']}, max_depth={best_rf_params['max_depth']}, "
      f"min_samples_split={best_rf_params['min_samples_split']}, min_samples_leaf={best_rf_params['min_samples_leaf']}")
rf_opt_metrics = evaluate_optimized_model(best_rf, "Random Forest optimizado")

print("\nExtra Trees optimizado")
print(f"Configuración: n_estimators={best_et_params['n_estimators']}, max_depth={best_et_params['max_depth']}, "
      f"min_samples_split={best_et_params['min_samples_split']}, min_samples_leaf={best_et_params['min_samples_leaf']}")
et_opt_metrics = evaluate_optimized_model(best_et, "Extra Trees optimizado")

print("\nBagging con Regresión Logística optimizado")
print(f"Configuración: n_estimators={best_bagging_lr_params['n_estimators']}, max_samples={best_bagging_lr_params['max_samples']}, "
      f"max_features={best_bagging_lr_params['max_features']}, bootstrap={best_bagging_lr_params['bootstrap']}, "
      f"bootstrap_features={best_bagging_lr_params['bootstrap_features']}")
bagging_lr_opt_metrics = evaluate_optimized_model(best_bagging_lr, "Bagging con Regresión Logística optimizado")

# Tabla comparativa de métricas
optimized_comparison = pd.DataFrame({
    'Modelo': ['Random Forest optimizado', 'Extra Trees optimizado', 'Bagging-LR optimizado'],
    'Accuracy Train': [rf_opt_metrics['accuracy_train'], et_opt_metrics['accuracy_train'], bagging_lr_opt_metrics['accuracy_train']],
    'Accuracy Test': [rf_opt_metrics['accuracy_test'], et_opt_metrics['accuracy_test'], bagging_lr_opt_metrics['accuracy_test']],
    'Precision': [rf_opt_metrics['precision'], et_opt_metrics['precision'], bagging_lr_opt_metrics['precision']],
    'Recall': [rf_opt_metrics['recall'], et_opt_metrics['recall'], bagging_lr_opt_metrics['recall']],
    'F1-Score': [rf_opt_metrics['f1'], et_opt_metrics['f1'], bagging_lr_opt_metrics['f1']]
})

print("\nTabla 3.5. Métricas de rendimiento de modelos optimizados:")
display(optimized_comparison)

# Evaluación de modelos optimizados

Random Forest optimizado
Configuración: n_estimators=25, max_depth=5, min_samples_split=2, min_samples_leaf=2


,Modelo,Accuracy Train,Accuracy Test,Precision,Recall,F1-Score
0,Random Forest optimizado,1.0,0.994152,0.990741,1.0,0.995349


,Predicción Maligno (0),Predicción Benigno (1)
Maligno (0),63,1
Benigno (1),0,107



Extra Trees optimizado
Configuración: n_estimators=25, max_depth=5, min_samples_split=10, min_samples_leaf=1


,Modelo,Accuracy Train,Accuracy Test,Precision,Recall,F1-Score
0,Extra Trees optimizado,0.997487,1.0,1.0,1.0,1.0


...


Bagging con Regresión Logística optimizado
Configuración: n_estimators=10, max_samples=0.5, max_features=1.0, bootstrap=False, bootstrap_features=False


,Modelo,Accuracy Train,Accuracy Test,Precision,Recall,F1-Score
0,Bagging con Regresión Logística optimizado,1.0,1.0,1.0,1.0,1.0


,Predicción Maligno (0),Predicción Benigno (1)
Maligno (0),64,0
Benigno (1),0,107



Tabla 3.5. Métricas de rendimiento de modelos optimizados:


,Modelo,Accuracy Train,Accuracy Test,Precision,Recall,F1-Score
0,Random Forest optimizado,1.000000,0.994152,0.990741,1.0,0.995349
1,Extra Trees optimizado,0.997487,1.000000,1.000000,1.0,1.000000
2,Bagging-LR optimizado,1.000000,1.000000,1.000000,1.0,1.000000


In [9]:
# Configuraciones originales vs optimizadas
configs = {
    'Modelo': ['Random Forest', 'Random Forest (opt)', 
              'Extra Trees', 'Extra Trees (opt)',
              'Bagging con LR', 'Bagging con LR (opt)'],
    'n_estimators': [100, 25, 100, 25, 100, 10],
    'max_depth': ['None', 5, 'None', 5, 'N/A', 'N/A'],
    'min_samples_split': [2, 2, 2, 10, 'N/A', 'N/A'],
    'min_samples_leaf': [1, 2, 1, 1, 'N/A', 'N/A'],
    'max_samples': ['N/A', 'N/A', 'N/A', 'N/A', 1.0, 0.5],
    'max_features': ['N/A', 'N/A', 'N/A', 'N/A', 1.0, 1.0],
    'bootstrap': ['True', 'True', 'True', 'True', 'True', 'False'],
    'bootstrap_features': ['False', 'False', 'False', 'False', 'False', 'False'],
    'Accuracy Test': [93.56, 99.42, 97.08, 100.0, 94.15, 100.0],
    'Reducción Complejidad': ['N/A', '~75%', 'N/A', '~75%', 'N/A', '~90%']
}

# Crear DataFrame
comparison_df = pd.DataFrame(configs)

# Mostrar tabla
print("Tabla 3.6. Comparación de configuraciones: modelos originales vs optimizados")
display(comparison_df)

Tabla 3.6. Comparación de configuraciones: modelos originales vs optimizados


,Modelo,n_estimators,max_depth,min_samples_split,min_samples_leaf,max_samples,max_features,bootstrap,bootstrap_features,Accuracy Test,Reducción Complejidad
0,Random Forest,100,None,2,1,N/A,N/A,True,False,93.56,N/A
1,Random Forest (opt),25,5,2,2,N/A,N/A,True,False,99.42,~75%
2,Extra Trees,100,None,2,1,N/A,N/A,True,False,97.08,N/A
3,Extra Trees (opt),25,5,10,1,N/A,N/A,True,False,100.00,~75%
4,Bagging con LR,100,N/A,N/A,N/A,1.0,1.0,True,False,94.15,N/A
5,Bagging con LR (opt),10,N/A,N/A,N/A,0.5,1.0,False,False,100.00,~90%


Los modelos optimizados muestran resultados variados tras la nueva partición de datos (70-30): Extra Trees y Bagging con LR mantienen el rendimiento perfecto (100% de exactitud), mientras que Random Forest muestra una ligera reducción a 99.42%. En cuanto al comportamiento en entrenamiento, Extra Trees presenta una exactitud de 99.75% frente al 100% anterior, lo que podría indicar menor tendencia al sobreajuste.

Esta comparación actualizada revela varios aspectos interesantes:

1. La reducción en el número de árboles/estimadores ahora es del 75% para Random Forest y Extra Trees (de 100 a 25), menor que la reducción del 90% observada previamente, sugiriendo que con más datos de prueba se requiere mayor complejidad para mantener el rendimiento.

2. Para Random Forest y Extra Trees, la profundidad máxima óptima es ahora de 5 niveles (más restrictiva que los 10 niveles anteriores), lo que indica que modelos más simples pueden capturar eficazmente los patrones tras la nueva partición.

3. Para Bagging con LR, la proporción de muestras se mantiene al 50%, pero ahora el bootstrap está desactivado, lo que representa un cambio en la estrategia de muestreo para adaptarse a la nueva distribución de datos.

4. Con la nueva partición 70-30, que proporciona una evaluación más robusta, Extra Trees destaca particularmente al mantener el 100% de exactitud a pesar de la reducción en complejidad, mientras que Random Forest experimenta una mínima degradación.

La capacidad de mantener un rendimiento excepcional con configuraciones simplificadas, incluso con una evaluación más exigente (conjunto de prueba mayor), confirma que los patrones en este conjunto de datos son altamente consistentes y bien definidos tras el preprocesamiento y selección de características adecuados. No obstante, la necesidad de más estimadores revela la importancia de equilibrar adecuadamente la complejidad del modelo con la proporción de datos destinados a entrenamiento y prueba.

In [10]:
# Crear directorio para los modelos optimizados si no existe
os.makedirs('data/optimizados', exist_ok=True)

# Guardar los modelos optimizados y datos necesarios
optimized_data = {
    'random_forest': best_rf,
    'extra_trees': best_et,
    'bagging_lr': best_bagging_lr,
    'X_train': X_train_model,
    'X_test': X_test_model,
    'y_train': y_train_model,
    'y_test': y_test_model,
    'feature_names': X_train_model.columns.tolist()
}

# Guardar en archivo
with open('data/optimizados/modelos_optimizados.pkl', 'wb') as f:
    pickle.dump(optimized_data, f)

print("Modelos optimizados y datos guardados correctamente para la fase de explicabilidad.")
print(f"Archivo guardado en: data/optimizados/modelos_optimizados.pkl")
print(f"Contiene {len(optimized_data['feature_names'])} características y {len(optimized_data['y_test'])} muestras de prueba.")

Modelos optimizados y datos guardados correctamente para la fase de explicabilidad.
Archivo guardado en: data/optimizados/modelos_optimizados.pkl
Contiene 18 características y 171 muestras de prueba.


# <font color="#000000"> 5. Conclusiones</font><a id="section5"></a>

A lo largo de este trabajo hemos aplicado algoritmos de Random Forest y otras técnicas ensemble al conjunto de datos "Breast Cancer Wisconsin (Diagnostic)" para la clasificación de tumores mamarios. El análisis exploratorio inicial mostró características con alto valor discriminativo, principalmente relacionadas con concavidad y puntos cóncavos, que presentaron diferencias estadísticamente significativas entre clases. La presencia de distribuciones no normales, correlaciones elevadas entre variables de tamaño y valores atípicos determinó nuestra estrategia de preprocesamiento adaptativo.

La implementación de transformaciones específicas según el nivel de asimetría de cada variable, seguida por una normalización robusta por clase, mejoró significativamente las propiedades estadísticas de los datos, reduciendo la asimetría media en un 84.48% y la curtosis media en un 99.48%. El análisis de redundancia mediante clustering jerárquico complementado por métodos de selección (Random Forest, Extra Trees, ANOVA F-value e Información Mutua) nos permitió reducir la dimensionalidad en un 43.3%, conservando las variables con mayor poder discriminativo.

Los resultados de modelado mostraron un rendimiento perfecto (100% de exactitud) para los modelos ensemble aplicados a datos preprocesados, superando a sus equivalentes con datos originales (92-96%). Esta diferencia cuantifica el impacto de nuestras técnicas de preprocesamiento y selección de variables. No obstante, este rendimiento perfecto requiere consideración metodológica, pues podría sugerir cierta adaptación específica a la partición de prueba utilizada. La optimización de hiperparámetros consiguió reducir la complejidad de los modelos en un 90-95% sin comprometer su capacidad predictiva, lo que evidencia que la estructura subyacente del problema es más simple de lo inicialmente estimado.

Estos resultados destacan el valor de combinar técnicas de preprocesamiento adaptativo con algoritmos ensemble para problemas de clasificación biomédica. Las variables relacionadas con irregularidades morfológicas (concavidad) demuestran ser determinantes en la identificación de malignidad, alineándose con el conocimiento médico existente sobre patología tumoral. El trabajo realizado proporciona una metodología replicable para el análisis de datos biomédicos con aplicación potencial en sistemas de apoyo al diagnóstico.